<a href="https://colab.research.google.com/github/HeeJin09/colab_test/blob/colab/UNet_model_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd /content/drive/My Drive/test

In [ ]:
ls

In [ ]:
#from google.colab import files
#src = list(files.upload().values())[0]
#open('model.py','wb').write(src)
#open('data.py','wb').write(src)
from model import *
from data import *
import tensorflow as tf
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

data_gen_args = dict(rotation_range=0.25,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'datasets','input','label',data_gen_args,save_to_dir = None)

model = unet()
def train():
    start_time = time.time()
    model_checkpoint = ModelCheckpoint('covidtest.hdf5', monitor='loss',verbose=1, save_best_only=True)
    print('total time(min):', (time.time()-start_time)/60)
def test():
    model.load_weights('./drive/MyDrive/datasets/covid_test.hdf5')
    testGene = testmyGenerator("./drive/MyDrive/datasets/test_image")
    results = model.predict_generator(testGene,3,verbose=1)
    saveResult("./drive/MyDrive/test/datasets/result",results)

train()

In [ ]:
from model import *
from data import *
import tensorflow as tf
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

data_gen_args = dict(rotation_range=0.25,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'datasets','input','label',data_gen_args,save_to_dir = None)

model = unet()
def train():
    start_time = time.time()
    model_checkpoint = ModelCheckpoint('covid_test.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene,steps_per_epoch=300,epochs=50,callbacks=[model_checkpoint])
    print('total time(min):', (time.time()-start_time)/60)
def test():
    model.load_weights('./covidtest.hdf5')
    testGene = testmyGenerator("./datasets/test_image")
    results = model.predict_generator(testGene,3,verbose=1)
    saveResult("./datasets/result",results)

#train()
test()

In [ ]:
import cv2
import os
import numpy as np
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend

def image_blending():
    original_path = './datasets/test_image'
    segmentation_path = './datasets/result'

    img1 = cv2.imread(os.path.join(original_path,'input__0.jpg'))
    img2 = cv2.imread(os.path.join(segmentation_path, '0_predict.png'))
    img2[:, :, 1] = 0
    img2[:, :, 2] = 0
    a=0.5
    b=0.5
    dst = cv2.addWeighted(img1, a, img2, b, 0)
    # cv2.imshow('dst',dst)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite(os.path.join(segmentation_path, 'image_blending_image1.png'), dst)

image_blending()

In [ ]:
from IPython.display import Image
Image('./datasets/result/image_blending_image1.png')